#Ансамбли моделей машинного обучения. Часть 2.
Цель лабораторной работы: изучение ансамблей моделей машинного обучения.

##Требования к отчету:
Отчет по лабораторной работе должен содержать:

1. титульный лист;
2. описание задания;
3. текст программы;
4. экранные формы с примерами выполнения программы.

##Задание:
1. Выберите набор данных (датасет) для решения задачи классификации или регресии.

2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.

3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.

4. Обучите следующие ансамблевые модели:
 - одну из моделей группы стекинга.
 - модель многослойного персептрона. По желанию, вместо библиотеки scikit-learn возможно использование библиотек TensorFlow, PyTorch или других аналогичных библиотек.
 - двумя методами на выбор из семейства МГУА (один из линейных методов COMBI / MULTI + один из нелинейных методов MIA / RIA) с использованием библиотеки gmdh.
 - В настоящее время библиотека МГУА не позволяет решать задачу классификации !!!


5. Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей.


###Установим библиотеку gmdh

In [2]:
!pip install gmdh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.3/875.3 kB 6.8 MB/s eta 0:00:00


###Выберем набор данных, проверим на наличие пропусков и обработаем их, проведем кодирование категориальных признаков, разделим данные на признаки и целевую переменную, разделим на обучающую и тестовую выборки, масштабируем данные.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import gmdh

# Загрузка данных
df = pd.read_csv('nasa.csv')

# Проверка на пропуски и их обработка
df = df.dropna()

# Кодирование категориальных признаков
le = LabelEncoder()
df['Orbiting Body'] = le.fit_transform(df['Orbiting Body'])

# Удаление столбцов с датами и строковыми значениями
columns_to_remove = ['Equinox','Close Approach Date', 'Orbit Determination Date', 'Perihelion Time', 'Epoch Date Close Approach', 'Epoch Osculation', 'Orbiting Body']
df = df.drop(columns=columns_to_remove)

# Разделение данных на признаки и целевую переменную
X = df.drop(columns=['Hazardous'])
y = df['Hazardous']

# Разделение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
gmdh_x_train, gmdh_x_test, gmdh_y_train, gmdh_y_test = gmdh.split_data(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##Обучим модель стекинга

- модель стекинга

In [7]:
# Обучение модели стекинга
estimators = [
    ('lr', LogisticRegression()),
    ('dt', DecisionTreeClassifier())
]
stacking_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
stacking_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('dt', DecisionTreeClassifier())],
                   final_estimator=LogisticRegression())

- модель многослойного персептрона

In [8]:
# Обучение модели многослойного персептрона
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(random_state=42)

- Обучение модели МГУА. Обучение линейного метода COMBI или MULTI

In [9]:
# Обучение модели МГУА
# Обучение линейного метода COMBI или MULTI
linear_gmdh = gmdh.Combi()
linear_gmdh.fit(gmdh_x_train, gmdh_y_train,verbose=1, limit=3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LEVEL 1  [=========================] 100% :00s] (32 combinations) error=209.032709
LEVEL 2  [=========================] 100% :00s] (496 combinations) error=151.583488
LEVEL 3  [=========================] 100% :00s] (4960 combinations) error=149.003227


- Обучение модели МГУА. Обучение нелинейного метода MIA

In [10]:
nonlinear_gmdh = gmdh.Mia()
nonlinear_gmdh.fit(gmdh_x_train, gmdh_y_train)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


###Оценка моделей

In [24]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred_stacking = stacking_clf.predict(X_test)
y_pred_mlp = mlp.predict(X_test)
y_pred_linear_gmdh = linear_gmdh.predict(gmdh_x_test)
y_pred_nonlinear_gmdh = nonlinear_gmdh.predict(gmdh_x_test)

# Преобразование булевых значений в числовой формат
y_test_numeric = y_test.astype(int)
gmdh_y_test_numeric = gmdh_y_test.astype(int)
y_pred_stacking_numeric = y_pred_stacking.astype(int)
y_pred_mlp_numeric = y_pred_mlp.astype(int)
y_pred_linear_gmdh_numeric = y_pred_linear_gmdh.astype(int)
y_pred_nonlinear_gmdh_numeric = y_pred_nonlinear_gmdh.astype(int)

# Расчет метрик
stacking_mse = mean_squared_error(y_test_numeric, y_pred_stacking_numeric)
mlp_mse = mean_squared_error(y_test_numeric, y_pred_mlp_numeric)
linear_gmdh_mse = mean_squared_error(gmdh_y_test_numeric, y_pred_linear_gmdh_numeric)
nonlinear_gmdh_mse = mean_squared_error(gmdh_y_test_numeric, y_pred_nonlinear_gmdh_numeric)

print(f"Stacking MSE: {stacking_mse}")
print(f"MLP MSE: {mlp_mse}")
print(f"Linear GMDH MSE: {linear_gmdh_mse}")
print(f"Nonlinear GMDH MSE: {nonlinear_gmdh_mse}")


print("\nAccuracy:")
# Можно также рассчитать другие метрики, такие как точность
stacking_accuracy = accuracy_score(y_test_numeric, y_pred_stacking_numeric)
mlp_accuracy = accuracy_score(y_test_numeric, y_pred_mlp_numeric)
linear_gmdh_accuracy = accuracy_score(gmdh_y_test_numeric, y_pred_linear_gmdh_numeric)
nonlinear_gmdh_accuracy = accuracy_score(gmdh_y_test_numeric, y_pred_nonlinear_gmdh_numeric)

print(f"Stacking Accuracy: {stacking_accuracy}")
print(f"MLP Accuracy: {mlp_accuracy}")
print(f"Linear GMDH Accuracy: {linear_gmdh_accuracy}")
print(f"Nonlinear GMDH Accuracy: {nonlinear_gmdh_accuracy}")

Stacking MSE: 0.0042643923240938165
MLP MSE: 0.009594882729211088
Linear GMDH MSE: 0.13127001067235858
Nonlinear GMDH MSE: 0.10138740661686232

Accuracy:
Stacking Accuracy: 0.9957356076759062
MLP Accuracy: 0.990405117270789
Linear GMDH Accuracy: 0.8687299893276414
Nonlinear GMDH Accuracy: 0.8986125933831377


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
